## AutoGen Conversation Patterns: Nested Chats

*[Coding along with the Udemy online course [Mastering AutoGen: Building Multi-Agent Systems](https://www.udemy.com/course/autogen-agent-systems/?couponCode=MTST7102224A2) by Paulo Dichone]*

In [1]:
from autogen import AssistantAgent, UserProxyAgent
import pandas as pd
from pprint import pprint

The Nested Chat Pattern allows the user to encapsulate complex workflows in one single conversational interface.

<img src="../assets/images/nested-chat.jpg" width="80%" />

In [2]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [3]:
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0.4,
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [4]:
# define the writer agent
writer = AssistantAgent(
    name="Writer",
    llm_config=llm_config,
    # the more detiled you are with the system message, the better the agent will work
    system_message="""
    You are a professional writer, known for your insightful and engaging product reviews.
    You transform technical details into compelling narratives.
    You should improve the quality of the content based on the feedback from the user.
    """,
)

In [5]:
# define the user proxy agent
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "my_code",
        "use_docker": False,
    },
)

In [6]:
# define the critic agent
critic = AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message="""
    You are a critic, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    """,
)

In [7]:
# function to generate reflection message
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...")
    return f"Reflect and provide critique on the following review. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

In [8]:
# register nested chats with the user proxy agent
user_proxy.register_nested_chats(
    [
        {
            "recipient": critic,
            "message": reflection_message,
            "summary_method": "last_msg",
            "max_turns": 1,
        }
    ],
    trigger=writer,
)

In [9]:
# define the task
task = """Write a detailed and engaging product review for the new Meta VR headset."""

In [10]:
# start the nested chat
res = user_proxy.initiate_chat(
    recipient=writer, message=task, max_turns=2, summary_method="last_msg"
)

User (to Writer):

Write a detailed and engaging product review for the new Meta VR headset.

--------------------------------------------------------------------------------
Writer (to User):

**Product Review: Meta Quest Pro VR Headset**

In the ever-evolving landscape of virtual reality, the Meta Quest Pro VR headset emerges as a beacon of innovation, promising to redefine our immersive experiences. With its sleek design, advanced technology, and a plethora of features, this headset is not just a gadget; it’s a portal to new dimensions. Let’s dive into what makes the Meta Quest Pro a standout choice for both casual users and hardcore VR enthusiasts.

**Design and Comfort: A New Standard**

Right out of the box, the Meta Quest Pro impresses with its modern aesthetic. The headset boasts a streamlined profile that feels both futuristic and approachable. Weighing in at a surprisingly light 600 grams, it’s designed for extended wear without compromising comfort. The adjustable straps and

In [11]:
pprint(res)

ChatResult(chat_id=None,
           chat_history=[{'content': 'Write a detailed and engaging product '
                                     'review for the new Meta VR headset.',
                          'name': 'User',
                          'role': 'assistant'},
                         {'content': '**Product Review: Meta Quest Pro VR '
                                     'Headset**\n'
                                     '\n'
                                     'In the ever-evolving landscape of '
                                     'virtual reality, the Meta Quest Pro VR '
                                     'headset emerges as a beacon of '
                                     'innovation, promising to redefine our '
                                     'immersive experiences. With its sleek '
                                     'design, advanced technology, and a '
                                     'plethora of features, this headset is '
                             